In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)


In [2]:
def collect_data(filepath):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))
    df=pd.read_json(all_files[0], lines=True)
    
    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        df_single =  pd.read_json(datafile, lines=True)
        df = pd.concat([df,df_single],ignore_index=True)     
    return df


In [3]:
songdf=collect_data('data/song_data')
logdf=collect_data('data/log_data')# store all the song_data into one dataframe

songdf.head()

101 files found in data/song_data
30 files found in data/log_data


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
1,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
2,ARB29H41187B98F0EF,41.88415,Chicago,-87.63241,Terry Callier,342.56934,1,SOGNCJP12A58A80271,Do You Finally Need A Friend,1972
3,ARGIWFO1187B9B55B7,NaN,,NaN,Five Bolt Main,225.09669,1,SOPSWQW12A6D4F8781,Made Like This (Live),0
4,ARH4Z031187B9A71F2,40.73197,"Newark, NJ",-74.17418,Faye Adams,156.39465,1,SOVYKGO12AB0187199,Crazy Mixed Up World,1961


In [12]:
logdf.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [22]:
results_artist=set(songdf['artist_name']).intersection(set(logdf['artist']))
results_songs=set(songdf['title']).intersection(set(logdf['song']))


In [25]:
print("There are {} common songs in log and song datasets which are \n{} ".format(len(results_songs),results_songs))
print("There are {} common artists in log and song datasets which are \n{} ".format(len(results_artist),results_artist))

There are 2 common songs in log and song datasets which are 
{'Intro', 'Setanta matins'} 
There are 11 common artists in log and song datasets which are 
{'Gwen Stefani', 'Lupe Fiasco', 'Lionel Richie', 'Tom Petty', 'Trafik', 'Gob', 'Line Renaud', 'Blue Rodeo', 'Jimmy Wakely', 'Sophie B. Hawkins', 'Elena'} 


In [30]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

[]
